In [216]:
import pandas as pd

In [217]:
def remover_espacos_colunas(df):
    for coluna in df.columns:
        if isinstance(coluna, str):  
            novo_nome = coluna.strip().replace(' ', '')  
            df.rename(columns={coluna: novo_nome}, inplace=True)  
            
    return df

In [218]:
def lista_colunas_unicas(dicionario_dataframes):
    colunas_unicas = set()
    for df_nome, df in dicionario_dataframes.items():
        colunas_unicas.update([col.replace('.1', '').strip() for col in df.columns])
    colunas_unicas.discard("ESTUDANTE")
    return list(colunas_unicas)
def extrair_elementos_para_dict(lista):
    dict_resultante = {}
    for idx, dic in enumerate(lista, start=1):
        dict_resultante[idx] = dic
    return dict_resultante
def extrair_abas_excel(nome_arquivo):
    xls = pd.ExcelFile(nome_arquivo)
    abas_dict = {}
    for aba_nome in xls.sheet_names:
        df = xls.parse(aba_nome)
        aba_nome = aba_nome.strip()
        abas_dict[aba_nome] = remover_espacos_colunas(df)
    return abas_dict
def gerar_alunos(lista_de_listas):
    alunos_unicos = set()
    for lista in lista_de_listas:
        alunos_unicos.update(lista)
    return list(alunos_unicos)

In [219]:
def calcular_participacao_aluno(df, flag_atraso):
    df_alvo = pd.DataFrame(df.iloc[:, 1:16]==flag_atraso)
    return df_alvo.replace({True: 1, False: 0}).sum(axis=1)
def calcular_participacao_materia(df, flag_atraso):
    df_alvo = pd.DataFrame(df.iloc[0:, 1:]==flag_atraso).sort_index(axis=1)
    return df_alvo.replace({True: 1, False: 0}).sum(axis=0).to_dict()
    

# Extração e processamento da frequencia das turmas 

In [220]:
path_frequencia_marco_2023 = 'data/raw/Frequencia_Marco_2023.xlsx'  
turmas_frequencia_marco_2023 = extrair_abas_excel(path_frequencia_marco_2023)
 

In [221]:
DISCIPLINAS = lista_colunas_unicas(turmas_frequencia_marco_2023)

In [222]:
DISCIPLINAS

['LITERATURA',
 'INTERTEXTO',
 'GEOGRAFIA',
 'BIOLOGIA',
 'SOCIOLOGIA',
 'FÍSICA',
 'QUÍMICA',
 'HISTÓRIA',
 'REDAÇÃO',
 'MATEMÁTICA',
 'FILOSOFIA',
 'PORTUGUÊS']

In [223]:
turmas_frequencia_marco_2023['Sala A - Semana 1'].columns

Index(['ESTUDANTE', 'FÍSICA', 'SOCIOLOGIA', 'PORTUGUÊS', 'QUÍMICA',
       'MATEMÁTICA', 'HISTÓRIA', 'GEOGRAFIA', 'LITERATURA', 'MATEMÁTICA.1',
       'BIOLOGIA', 'REDAÇÃO', 'INTERTEXTO'],
      dtype='object')

## Gerando panorama das disciplinas

### Buscando todas as disciplinas do mês

In [224]:
salas = ['A','B']
semanas = [1,2,3]
arquivos = list(turmas_frequencia_marco_2023.keys())
resultado_disciplinas = {}
cod = 1

In [225]:
dados_disciplinas = {}
for semana in semanas:
    for sala in salas:
        chave = f"{semana:02d}_{sala}"
        arquivo = f'Sala {sala} - Semana {semana}'
        dados_disciplinas[chave] = {'sala': sala, 'semana': semana, 'arquivo': arquivo}

In [226]:
resultados = []

In [227]:
for id in list(dados_disciplinas.keys()):
    arquivo_base =  dados_disciplinas[id]['arquivo']
    for disciplina in DISCIPLINAS:
        discp_semana = {}  # Crie um novo dicionário para cada disciplina
        disciplina_extra = disciplina + '.1'
        disciplinas_semana = list(turmas_frequencia_marco_2023[arquivo_base].columns)
        if disciplina in disciplinas_semana:
            qtd_aulas = 1
            atraso15 = calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], '0*')[disciplina]
            atraso30 = calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], '0**')[disciplina]
            faltas = calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], 0)[disciplina]
            presencas = calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], 1)[disciplina]
            if disciplina_extra in disciplinas_semana:
                qtd_aulas = qtd_aulas + 1
                atraso15 = (atraso15 + calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], '0*')[disciplina_extra])
                atraso30 = (atraso30 + calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], '0**')[disciplina_extra])
                faltas = (faltas + calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], 0)[disciplina_extra])
                presencas = (presencas + calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], 1)[disciplina_extra])         
        else:
            qtd_aulas = 0
            atraso15 = 0
            atraso30 = 0
            faltas = 0
            presencas = 0
        discp_semana[cod] = {
            'ID': id,
            'SEMANA': dados_disciplinas[id]['semana'],
            'SALA': dados_disciplinas[id]['sala'],
            'DISCIPLINA': disciplina,
            'QTD_AULAS_SEMANA': qtd_aulas,
            'PRESENÇAS': presencas,
            'FALTAS': faltas,
            'ATRASO15': atraso15,
            'ATRASO30': atraso30
        }
        resultados.append(discp_semana)
        cod += 1

In [228]:
tam_base = len(resultados)
base_final = {}
for num in range(1,tam_base+1):
    base_final[num] = extrair_elementos_para_dict(resultados)[num][num]
base_disciplinas = pd.DataFrame(base_final).T    
    

In [236]:
# base_disciplinas.to_csv('situacao_disciplinas.csv',index=False)

## Gerando base de alunos

In [230]:
alunos_sala_a = []
alunos_sala_b = []   
for semana in range(1, 4):
    alunos_sala_a.append(list(turmas_frequencia_marco_2023[f'Sala A - Semana {semana}']['ESTUDANTE']))
    alunos_sala_b.append(list(turmas_frequencia_marco_2023[f'Sala B - Semana {semana}']['ESTUDANTE']))
    
    

In [231]:
alunos_sala_a = gerar_alunos(alunos_sala_a)
alunos_sala_b = gerar_alunos(alunos_sala_b)

In [232]:
def gerar_presenca_alunos(turma,alunos_turma):
    base_alunos_sala_parcial = {}
    id = 1 
    for aluno in alunos_turma:
        for semana in range(1, 4):
            try:
                df_semana = turmas_frequencia_marco_2023[f'Sala {turma} - Semana {semana}'].query(f"ESTUDANTE == '{aluno}'")
                base_alunos_sala_parcial[id] = {
                    'ESTUDANTE': aluno,
                    'TURMA': turma,
                    'SEMANA': semana,
                    'PRESENCA': int(calcular_participacao_aluno(df_semana, 1)), 
                    'FALTAS': int(calcular_participacao_aluno(df_semana, 0)),
                    'ATRASOS15': int(calcular_participacao_aluno(df_semana, '0*')),
                    'ATRASOS30': int(calcular_participacao_aluno(df_semana, '0**'))
                }
            except:
                base_alunos_sala_parcial[id] = {
                    'ESTUDANTE': aluno,
                    'TURMA': turma,
                    'SEMANA': semana,
                    'PRESENCA': -1,
                    'FALTAS': -1,
                    'ATRASOS15': -1,
                    'ATRASOS30': -1
                }
            id += 1
    return pd.DataFrame(base_alunos_sala_parcial).T

In [233]:
gerar_presenca_alunos('A',alunos_sala_a)

,ESTUDANTE,TURMA,SEMANA,PRESENCA,FALTAS,ATRASOS15,ATRASOS30
1,Vinícius Rocha Almeida,A,1,1,11,0,0
2,Vinícius Rocha Almeida,A,2,6,4,0,0
3,Vinícius Rocha Almeida,A,3,2,8,1,0
4,Maria Luiza da Silva,A,1,0,10,1,1
5,Maria Luiza da Silva,A,2,2,6,2,0
...,...,...,...,...,...,...,...
230,Larissa Costa Silva,A,2,0,9,1,0
231,Larissa Costa Silva,A,3,3,7,1,0
232,Letícia Rodrigues Santos,A,1,1,6,3,2
233,Letícia Rodrigues Santos,A,2,6,2,2,0


In [234]:
gerar_presenca_alunos('B',alunos_sala_b)

,ESTUDANTE,TURMA,SEMANA,PRESENCA,FALTAS,ATRASOS15,ATRASOS30
1,Amanda Almeida Lima,B,1,0,12,0,0
2,Amanda Almeida Lima,B,2,0,10,0,0
3,Amanda Almeida Lima,B,3,2,9,0,0
4,Camila Almeida Santos,B,1,2,10,0,0
5,Camila Almeida Santos,B,2,0,10,0,0
...,...,...,...,...,...,...,...
224,André Oliveira Santos,B,2,-1,-1,-1,-1
225,André Oliveira Santos,B,3,0,10,1,0
226,Vitória Santos Alves,B,1,0,11,1,0
227,Vitória Santos Alves,B,2,10,0,0,0
